## National Household Surveys 

Processing/Cleaning first try: tried creating bounding boxes using KULL and using with PDF Query but got bad results

Processing/Cleaning second try: using Tesseract to convert PDF into PNG and using KULL to create uzn file

Source: https://www.toronto.ca/city-government/data-research-maps/neighbourhoods-communities/ward-profiles/

In [1]:
import pandas as pd
%matplotlib inline
import numpy as np
import glob
import pdfquery

In [24]:
pdfs = glob.glob('wards/*.pdf')
pdfs

['wards/ward01.pdf',
 'wards/ward02.pdf',
 'wards/ward03.pdf',
 'wards/ward04.pdf',
 'wards/ward05.pdf',
 'wards/ward06.pdf',
 'wards/ward07.pdf',
 'wards/ward08.pdf',
 'wards/ward09.pdf',
 'wards/ward10.pdf',
 'wards/ward11.pdf',
 'wards/ward12.pdf',
 'wards/ward13.pdf',
 'wards/ward14.pdf',
 'wards/ward15.pdf',
 'wards/ward16.pdf',
 'wards/ward17.pdf',
 'wards/ward18.pdf',
 'wards/ward19.pdf',
 'wards/ward20.pdf',
 'wards/ward21.pdf',
 'wards/ward22.pdf',
 'wards/ward23.pdf',
 'wards/ward24.pdf',
 'wards/ward25.pdf',
 'wards/ward26.pdf',
 'wards/ward27.pdf',
 'wards/ward28.pdf',
 'wards/ward29.pdf',
 'wards/ward30.pdf',
 'wards/ward31.pdf',
 'wards/ward32.pdf',
 'wards/ward33.pdf',
 'wards/ward34.pdf',
 'wards/ward35.pdf',
 'wards/ward36.pdf',
 'wards/ward37.pdf',
 'wards/ward38.pdf',
 'wards/ward39.pdf',
 'wards/ward40.pdf',
 'wards/ward41.pdf',
 'wards/ward42.pdf',
 'wards/ward43.pdf',
 'wards/ward44.pdf']

## Try 1 - using PDFQuery and Kull

In [3]:
pdf = pdfquery.PDFQuery("wards/ward01.pdf")
pdf.load()

In [4]:
label = pdf.pq('LTTextLineHorizontal:contains("Income")')
label

[<LTTextLineHorizontal>, <LTTextLineHorizontal>, <LTTextLineHorizontal>, <LTTextLineHorizontal>, <LTTextLineHorizontal>, <LTTextLineHorizontal>, <LTTextLineHorizontal>, <LTTextLineHorizontal>, <LTTextLineHorizontal>, <LTTextLineHorizontal>, <LTTextLineHorizontal>, <LTTextLineHorizontal>, <LTTextLineHorizontal>]

In [5]:
label.text()

'2010 Household Income 2010Household Income Household Income - 2010 Low Income - 2010 Low Income* Low Income* Individual Income (15 years and over) - 2010 Composition of Income (15 years and over) - 2010 Income / Shelter Costs Average Household Income AverageHousehold Income AverageIndividual Income Composition of Income Household Income Total Income'

In [6]:
pdf = pdfquery.PDFQuery('wards/ward01.pdf')

query=[]

my_query = pdf.extract([
	('with_formatter', 'text'),
	('income', 'LTPage[pageid=\'1\'] LTTextLineHorizontal:overlaps_bbox("314,372,378,357")')
])

query.append(my_query)
query

[{'income': ''}]

## Try 2 - using Tesseract and Kull



Use ImageMagick to convert our PDF into PNG 

In [7]:

!convert -density 300 wards/ward01.pdf wards/output.png

Use glob to combine all the png files and remove the pngs that you don't need

In [8]:
pngs = glob.glob('wards/output-*.png')
pngs.remove('wards/output-1.png')
pngs.remove('wards/output-2.png')
pngs.remove('wards/output-3.png')
pngs.remove('wards/output-4.png')
pngs.remove('wards/output-5.png')
pngs.remove('wards/output-6.png')
pngs.remove('wards/output-7.png')
pngs.remove('wards/output-8.png')
pngs.remove('wards/output-9.png')
pngs.remove('wards/output-10.png')
pngs.remove('wards/output-11.png')
pngs.remove('wards/output-12.png')
pngs.remove('wards/output-13.png')
pngs.remove('wards/output-14.png')
pngs.remove('wards/output-15.png')
pngs.remove('wards/output-16.png')
pngs.remove('wards/output-17.png')
pngs.remove('wards/output-18.png')
pngs.remove('wards/output-19.png')
pngs.remove('wards/output-20.png')
pngs.remove('wards/output-21.png')

In [9]:
pngs

['wards/output-0.png']

Use tesseract and uzn file created from Kull to create a text file

In [10]:
for png in pngs:
    !cp output-0.uzn wards/output-0.uzn
    !mv wards/output-0.uzn {png[:-4]}.uzn
    !tesseract {png} {png[:-4]} -c tessedit_char_blacklist=ﬁﬂ -psm 6
    

Tesseract Open Source OCR Engine v3.05.01 with Leptonica
Warning. Invalid resolution 0 dpi. Using 70 instead.


Combine all text files using glob

In [11]:
texts = glob.glob('wards/output-*.txt')

Read in text and append its data into empty lists

In [12]:
ward_no = []
income = []
unemployment = []

for text in texts:
    with open(text) as myfile:
        lines = myfile.read().splitlines()
    ward_no.append(lines[0])
    income.append(lines[2])
    unemployment.append(lines[3])
    

In [13]:
#for some reason I can't seem to grab the unemployment data
lines

['Ward 1', '', '$66,001', '']

Create a data frame out of the empty lists

In [14]:
df = pd.DataFrame(np.column_stack([ward_no, income, unemployment]) , columns=['ward_no', 'income', 'unemployment_rate'])
df


,ward_no,income,unemployment_rate
0,Ward 1,"$66,001",


## Looping through all PDFs

In [53]:
for pdf in pdfs:
    !convert -density 300 pdf wards/output.png
    print(pdf)

convert: unable to open image 'pdf': No such file or directory @ error/blob.c/OpenBlob/3323.
convert: no decode delegate for this image format `' @ error/constitute.c/ReadImage/509.
convert: no images defined `wards/output.png' @ error/convert.c/ConvertImageCommand/3275.
wards/ward01.pdf
convert: unable to open image 'pdf': No such file or directory @ error/blob.c/OpenBlob/3323.
convert: no decode delegate for this image format `' @ error/constitute.c/ReadImage/509.
convert: no images defined `wards/output.png' @ error/convert.c/ConvertImageCommand/3275.
wards/ward02.pdf
convert: unable to open image 'pdf': No such file or directory @ error/blob.c/OpenBlob/3323.
convert: no decode delegate for this image format `' @ error/constitute.c/ReadImage/509.
convert: no images defined `wards/output.png' @ error/convert.c/ConvertImageCommand/3275.
wards/ward03.pdf
convert: unable to open image 'pdf': No such file or directory @ error/blob.c/OpenBlob/3323.
convert: no decode delegate for this ima

wards/ward29.pdf
convert: unable to open image 'pdf': No such file or directory @ error/blob.c/OpenBlob/3323.
convert: no decode delegate for this image format `' @ error/constitute.c/ReadImage/509.
convert: no images defined `wards/output.png' @ error/convert.c/ConvertImageCommand/3275.
wards/ward30.pdf
convert: unable to open image 'pdf': No such file or directory @ error/blob.c/OpenBlob/3323.
convert: no decode delegate for this image format `' @ error/constitute.c/ReadImage/509.
convert: no images defined `wards/output.png' @ error/convert.c/ConvertImageCommand/3275.
wards/ward31.pdf
convert: unable to open image 'pdf': No such file or directory @ error/blob.c/OpenBlob/3323.
convert: no decode delegate for this image format `' @ error/constitute.c/ReadImage/509.
convert: no images defined `wards/output.png' @ error/convert.c/ConvertImageCommand/3275.
wards/ward32.pdf
convert: unable to open image 'pdf': No such file or directory @ error/blob.c/OpenBlob/3323.
convert: no decode dele